<a href="https://colab.research.google.com/github/mmejdoubi/KEL/blob/master/manual_sliding_window.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import pandas as pd

In [0]:
## download dataset
! wget "https://onedrive.live.com/download?cid=ED9B50B3CA11E5C4&resid=ED9B50B3CA11E5C4%21138&authkey=AMa0qTOP6n7RDII" -O mad_subset.csv

--2019-01-31 16:52:53--  https://onedrive.live.com/download?cid=ED9B50B3CA11E5C4&resid=ED9B50B3CA11E5C4%21138&authkey=AMa0qTOP6n7RDII
Resolving onedrive.live.com (onedrive.live.com)... 13.107.42.13
Connecting to onedrive.live.com (onedrive.live.com)|13.107.42.13|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://6hjuna.dm.files.1drv.com/y4mOOmO9-9NXAZyKccLieKl3sJaVhY-DLGm1zH3RAlEhcR0yDI_ZhXPpDBp8d__5tDMEUabWqoS8ygOLOi1h0y9kG2tayP-cf1yoQ7T82SUbchsMkoIKBiN92yN22L2BrKF9vprmU3tJuNdNgrQTbp4-2xzr7kTWBwtP11B4qIbc_zfhQrdj-_lLmTbv3g197HWii_7X1bNSKSR2h-ghSoAeg/mad_subset.csv?download&psid=1 [following]
--2019-01-31 16:52:55--  https://6hjuna.dm.files.1drv.com/y4mOOmO9-9NXAZyKccLieKl3sJaVhY-DLGm1zH3RAlEhcR0yDI_ZhXPpDBp8d__5tDMEUabWqoS8ygOLOi1h0y9kG2tayP-cf1yoQ7T82SUbchsMkoIKBiN92yN22L2BrKF9vprmU3tJuNdNgrQTbp4-2xzr7kTWBwtP11B4qIbc_zfhQrdj-_lLmTbv3g197HWii_7X1bNSKSR2h-ghSoAeg/mad_subset.csv?download&psid=1
Resolving 6hjuna.dm.files.1drv.com (6hjuna.dm.files.1drv.

In [0]:
! head mad_subset.csv

,NUM_PERS,MONTANT,CANAL_TYPE,parsed_dt
16648,22160,2000,MBK,2018-08-18
16653,22160,2000,MBK,2018-11-12
16652,22160,2000,MBK,2018-10-16
16651,22160,2000,MBK,2018-09-11
16650,22160,2000,MBK,2018-09-25
16649,22160,1000,MBK,2018-08-18
16647,22160,1000,MBK,2018-07-09
16646,22160,1700,MBK,2018-07-19
16645,22160,500,MBK,2018-06-28


In [0]:
df = pd.read_csv("mad_subset.csv", index_col='NUM_PERS', parse_dates=['parsed_dt'])
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19277 entries, 22160 to 90622471
Data columns (total 4 columns):
Unnamed: 0    19277 non-null int64
MONTANT       19277 non-null int64
CANAL_TYPE    19277 non-null object
parsed_dt     19277 non-null datetime64[ns]
dtypes: datetime64[ns](1), int64(2), object(1)
memory usage: 753.0+ KB


In [0]:
print("nb_users => " + str(df.index.nunique()))
print("data start => " + str(df.parsed_dt.min()))
print("data end => " + str(df.parsed_dt.max())) 
## predire sum(montant) sur oct-nov, training sur data last 6, par semaines

nb_users => 2516
data start => 2017-01-01 00:00:00
data end => 2018-11-30 00:00:00


In [0]:
import datetime as dt
## decoupage en fenetres temporelles
cutoff_dt = pd.to_datetime('30/9/2018', dayfirst=True)
window_width = 7 ## detail jour
training_length = 180 ## 6 mois

## prediction window
prediction_length = 60

## nb de fenetres
window_count = int(training_length / window_width)
print('nbre fenetres historiques : ' + str(window_count))

nbre fenetres historiques : 25


In [0]:
## Df hist contient toute data >= debut historique, et <= cutoff
## df_hist va etre decoupe en window_count egales

train_start_dt = cutoff_dt - dt.timedelta(training_length)
training_data = df[(df["parsed_dt"] <= cutoff_dt) & (df["parsed_dt"] > train_start_dt)]

## alternative way todo direct in pandas (new in 0.24) => ts.between_time('0:45', '0:15')
## useful to calcul label 
prediction_end_dt = cutoff_dt + dt.timedelta(prediction_length)
prediction_data = df[(df["parsed_dt"] > cutoff_dt) & (df["parsed_dt"] <= prediction_end_dt)]

In [0]:
print("Total Training window start @ >> " + str(training_data['parsed_dt'].min()))
print("Total Training window end @ >> " + str(training_data['parsed_dt'].max()))
print("Prediction window end @ >> " + str(prediction_data['parsed_dt'].max()))

Total Training window start @ >> 2018-04-04 00:00:00
Total Training window end @ >> 2018-09-30 00:00:00
Prediction window end @ >> 2018-11-29 00:00:00


In [0]:
## boucle qui calcul indicateurs pour une seule fenetre des window_count, 
## params : 
## start_dt=
## end_dt= 
## starting_from cutoff_dt - (7d * iteration_number) à chaque fois
## in_df = training_data

def build_window(window_idx, in_df):
    ## Set index on date col to be able to use pandas loc fx
    in_df = in_df.set_index(in_df['parsed_dt'])
    ## window_end starting from cutoff and backward
    offset = int(window_idx * window_width)
    t_end = cutoff_dt - dt.timedelta(offset)
    t_start = cutoff_dt - dt.timedelta(offset + window_width)
    suffix_start = (cutoff_dt - t_start).days
    suffix_end = (cutoff_dt - t_end).days
    suffix_str = str(suffix_end) + '_' + str(suffix_start) + '_DAYS'
    ## slicing using df_w = df.loc['2002-1-1 01:00:00':'2002-1-1 04:00:00']
    
    ## populate window param dict
    # dictionary with mixed keys
    window_dict.update({window_idx: [t_start, t_end, suffix_str]})
     
    return window_dict

    ##print('window {} Start : {} End : {}'.format(window_idx, t_start, t_end))
    ## rename suffix : ecart en day entre cutoff, et t_start et t_end : agg_indic_7_15d

In [0]:
## stock window parametre as python dict
window_dict =  "global"
window_dict = {}

for i in range(0,window_count):
    build_window(i, training_data)

In [0]:
## loop over all periods
## todo :call a la fonction qui calcule les kpis

for i in range(0, len(window_dict)):
    print("window # {}, start : {}, End : {}, Variable Suffix : {}"
          .format(i, window_dict[i][0], window_dict[i][1], window_dict[i][2]))

window # 0, start : 2018-09-23 00:00:00, End : 2018-09-30 00:00:00, Variable Suffix : 0_7_DAYS
window # 1, start : 2018-09-16 00:00:00, End : 2018-09-23 00:00:00, Variable Suffix : 7_14_DAYS
window # 2, start : 2018-09-09 00:00:00, End : 2018-09-16 00:00:00, Variable Suffix : 14_21_DAYS
window # 3, start : 2018-09-02 00:00:00, End : 2018-09-09 00:00:00, Variable Suffix : 21_28_DAYS
window # 4, start : 2018-08-26 00:00:00, End : 2018-09-02 00:00:00, Variable Suffix : 28_35_DAYS
window # 5, start : 2018-08-19 00:00:00, End : 2018-08-26 00:00:00, Variable Suffix : 35_42_DAYS
window # 6, start : 2018-08-12 00:00:00, End : 2018-08-19 00:00:00, Variable Suffix : 42_49_DAYS
window # 7, start : 2018-08-05 00:00:00, End : 2018-08-12 00:00:00, Variable Suffix : 49_56_DAYS
window # 8, start : 2018-07-29 00:00:00, End : 2018-08-05 00:00:00, Variable Suffix : 56_63_DAYS
window # 9, start : 2018-07-22 00:00:00, End : 2018-07-29 00:00:00, Variable Suffix : 63_70_DAYS
window # 10, start : 2018-07-15 0

In [0]:
numeric_cols = ['MONTANT']
num_agg_fx = ['sum', 'mean', 'std']
group_by = ['CANAL_TYPE']